In [ ]:
from flask import Flask, render_template, request
import pandas as pd
import os
import traceback

app = Flask(__name__)

# Define CSV file path
csv_path = r"D:\Project food Genie\MERGED_FOOD_DATA_WITH_GRAMS.csv"

# Check if the CSV file exists
if not os.path.exists(csv_path):
    print(f"❌ ERROR: CSV file not found at {csv_path}")
else:
    try:
        print("🔄 Attempting to load CSV file...")
        df = pd.read_csv(csv_path, encoding="utf-8")
        print("✅ CSV file loaded successfully!")

        # Ensure necessary columns exist
        required_columns = ["food", "Caloric Value", "Protein", "Carbohydrates", "Fat"]

        if all(col in df.columns for col in required_columns):
            df["food"] = df["food"].fillna("")
            df["Caloric Value"] = pd.to_numeric(df["Caloric Value"], errors="coerce")
            df["Protein"] = pd.to_numeric(df["Protein"], errors="coerce")
            df["Carbohydrates"] = pd.to_numeric(df["Carbohydrates"], errors="coerce")
            df["Fat"] = pd.to_numeric(df["Fat"], errors="coerce")
            print("✅ Required nutrition columns found and processed!")
        else:
            print(f"❌ ERROR: Required columns missing in CSV! Available columns: {df.columns}")
            df = None  # Prevents app from running with an invalid DataFrame

    except Exception as e:
        print("❌ ERROR: Could not load CSV file.")
        traceback.print_exc()
        df = None  # Prevents app from running with an invalid DataFrame


@app.route("/", methods=["GET", "POST"])
def index():
    if df is None:
        return "❌ ERROR: Data not loaded. Please check the server logs.", 500

    results = None
    message = None

    if request.method == "POST":
        query = request.form.get("query", "").strip().lower()

        if not query:
            message = "Please enter a search term."
        else:
            try:
                # Filter results that contain the search term
                results = df[df["food"].str.lower().str.contains(query, na=False)]

                # Sort results by 'Caloric Value' in ascending order
                results = results.sort_values(by="Caloric Value", ascending=True)

                if results.empty:
                    message = "No matching food items found."
            except Exception as e:
                print("❌ ERROR during search:", e)
                traceback.print_exc()
                message = "An error occurred while searching. Please try again."

    return render_template("index.html", results=results, message=message)


if __name__ == "__main__":
    print("🚀 Starting Flask server...")
    app.run(debug=False, port=5001)


🔄 Attempting to load CSV file...
✅ CSV file loaded successfully!
✅ Required nutrition columns found and processed!
🚀 Starting Flask server...
 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5001
Press CTRL+C to quit
127.0.0.1 - - [08/Feb/2025 01:13:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [08/Feb/2025 01:13:58] "POST / HTTP/1.1" 200 -
